In [222]:
import pandas as pd 
import numpy as np
from haversine import haversine
import tqdm

In [223]:
res = pd.read_excel('./res_4.xlsx')
res.columns

Index(['Unnamed: 0', 'name', 'address', 'Latitude', 'Longitude', 'Category'], dtype='object')

In [224]:
res_hist = res[res['Category']=='유적지'].reset_index(drop=True)
res_can = res[res['Category']=='후보지'].reset_index(drop=True)

In [225]:
res_can.tail()

,Unnamed: 0,name,address,Latitude,Longitude,Category
17,283,후보지18,NaN,35.812754,127.152912,후보지
18,284,후보지19,NaN,35.814109,127.152695,후보지
19,285,후보지20,NaN,35.814923,127.153648,후보지
20,286,후보지21,NaN,35.815912,127.153967,후보지
21,287,후보지22,NaN,35.817325,127.153682,후보지


In [226]:
hist_n = [res_hist.iloc[i,1] for i in range(len(res_hist))]
can_n = ['후보지{}'.format(i+1) for i in range(len(res_can))]
# 유적지와 후보지 간의 m단위 거리
def dist_app(i):
    pin_lat = res_hist.iloc[i,3]
    pin_long = res_hist.iloc[i,4]
    res = []
    for j in range(len(res_can)):
        can_lat = res_can.iloc[j,3]
        can_long = res_can.iloc[j,4]
        dist_h = haversine((pin_lat,pin_long),(can_lat,can_long), unit='m')
        dist_r = np.round(dist_h,2)
        res.append((can_n[j],dist_r))
    return res

dist_r = [dist_app(i) for i in range(len(res_hist))]

In [227]:
dist_r[1][0]

('후보지1', 479.65)

In [228]:
hyangyo=pd.DataFrame(dist_r[0], columns=['name','dist'])
gyeonki = pd.DataFrame(dist_r[1], columns=['name','dist'])
jeondong = pd.DataFrame(dist_r[2], columns=['name','dist'])
omok = pd.DataFrame(dist_r[3], columns=['name','dist'])
cheong = pd.DataFrame(dist_r[4], columns=['name','dist'])
museum =pd.DataFrame(dist_r[5], columns=['name','dist'])

In [229]:
# 반경 335m 내 후보지만 추출(방문객이 5분 내로 도달할 수 있는 거리)
def sampling(df):
    df = df[df['dist']<=201].reset_index(drop=True)# 추출 후 인덱스 초기화
    return df

hyangyo = sampling(hyangyo)
gyeonki = sampling(gyeonki)
jeondong = sampling(jeondong)
omok = sampling(omok)
cheong = sampling(cheong)
museum = sampling(museum)

In [230]:
gyeonki

,name,dist
0,후보지4,163.78
1,후보지5,82.58
2,후보지6,152.10
3,후보지7,147.92


In [231]:
print('향교: {}'.format(hyangyo['dist'].count()))
print('경기전: {}'.format(gyeonki['dist'].count()))
print('전동성당: {}'.format(jeondong['dist'].count()))
print('오목대: {}'.format(omok['dist'].count()))
print('청연루: {}'.format(cheong['dist'].count()))
print('어진박물관: {}'.format(museum['dist'].count()))

향교: 4
경기전: 4
전동성당: 3
오목대: 3
청연루: 4
어진박물관: 3


In [232]:
len(hyangyo)

4

In [233]:
li = ['항교','경기전','전동성당','오목대','청연루','어진박물관']

def app(df,kew): 
    spot = pd.Series((kew for i in range(len(df))), name = 'spot')
    df = pd.concat([df,spot],axis=1,ignore_index=False) #ignore_index=true하면 컬럼명 날라감
    return df

hyangyo = app(hyangyo, li[0])
gyeonki = app(gyeonki, li[1])
jeondong = app(jeondong, li[2])
omok = app(omok, li[3])
cheong = app(cheong, li[4])
museum = app(museum, li[5])

In [234]:
hist = pd.concat([hyangyo,gyeonki,jeondong,omok,cheong,museum],axis=0, 
                 ignore_index=True)
hist

,name,dist,spot
0,후보지13,158.21,항교
1,후보지14,151.30,항교
2,후보지15,92.85,항교
3,후보지16,200.02,항교
4,후보지4,163.78,경기전
5,후보지5,82.58,경기전
6,후보지6,152.10,경기전
7,후보지7,147.92,경기전
8,후보지6,76.58,전동성당
9,후보지7,135.11,전동성당


In [235]:
hist.to_excel('./res_5.xlsx')

In [236]:
import folium as g
import json

In [237]:
res.columns

Index(['Unnamed: 0', 'name', 'address', 'Latitude', 'Longitude', 'Category'], dtype='object')

In [238]:
g_map = g.Map(location=[35.81563488182217, 127.15352107073629], zoom_start=18)

for i in range(len(res_hist)):
    lat = res_hist.loc[i]['Latitude']
    long = res_hist.loc[i]['Longitude']
    tit = res_hist.loc[i,'name']
    marker01 = g.CircleMarker([lat,long],color='black', tooltip=tit, radius=201).add_to(g_map) # 원크기 조정 필요

for j in range(len(res_can)):
    lat = res_can.loc[j, 'Latitude']
    long = res_can.loc[j, 'Longitude']
    tit = res_can.loc[j,'name']
    marker02 = g.Marker([lat,long],color='blue', tooltip=tit).add_to(g_map)
    
g_map

In [239]:
g_map.save('./vis_1.html')

In [240]:
hist.columns

Index(['name', 'dist', 'spot'], dtype='object')

In [241]:
hist_1  = hist.drop_duplicates(['name'], keep = 'first')

In [242]:
hist_1.reset_index(drop=True)

,name,dist,spot
0,후보지13,158.21,항교
1,후보지14,151.30,항교
2,후보지15,92.85,항교
3,후보지16,200.02,항교
4,후보지4,163.78,경기전
5,후보지5,82.58,경기전
6,후보지6,152.10,경기전
7,후보지7,147.92,경기전
8,후보지8,162.55,전동성당
9,후보지18,158.81,오목대


In [243]:
hist_2 = pd.merge(hist_1, res_can, 
                  how='inner', 
                  on=['name']).drop(['Unnamed: 0','address','Category'],
                                                                  axis=1)
hist_2['name'] = hist_2['name'].str.replace('후보지','').astype(int)
hist_2 = hist_2.sort_values(by=['name'],axis=0).reset_index(drop=True)
hist_2

,name,dist,spot,Latitude,Longitude
0,3,169.84,어진박물관,35.817231,127.150700
1,4,163.78,경기전,35.816704,127.149154
2,5,82.58,경기전,35.815412,127.148849
3,6,152.10,경기전,35.814038,127.149151
4,7,147.92,경기전,35.814193,127.150639
5,8,162.55,전동성당,35.813447,127.151297
6,11,107.76,청연루,35.811091,127.153401
7,12,81.00,청연루,35.811274,127.155030
8,13,158.21,항교,35.811462,127.156769
9,14,151.30,항교,35.811790,127.158172


In [244]:
g2_map = g.Map(location=[35.81563488182217, 127.15352107073629], zoom_start=17)

for i in range(len(hist_2)):
    lat = hist_2.loc[i]['Latitude']
    long = hist_2.loc[i]['Longitude']
    tit = hist_2.loc[i,'name']
    marker = g.Marker([lat,long], tooltip=tit).add_to(g2_map)
g2_map

In [245]:
hist_2.columns

Index(['name', 'dist', 'spot', 'Latitude', 'Longitude'], dtype='object')

In [246]:
# 음식점, 카페, 숙소 데이터 추출
res_ca = res[res['Category'].isin(['음식점','카페','숙소'])].reset_index(drop=True).drop(['Unnamed: 0'],axis=1).reset_index(drop=True)
res_ca.tail()

,name,address,Latitude,Longitude,Category
255,용이네게스트하우스게스트하우스,전라북도 전주시 완산구 어진길 122-6,35.816814,127.148102,숙소
256,하늘기와전통숙소,전라북도 전주시 완산구 전주천동로 54-8,35.811618,127.154851,숙소
257,취락당펜션,전라북도 전주시 완산구 동문길 33-19,35.818583,127.152733,숙소
258,알지비게스트하우스게스트하우스,전라북도 전주시 완산구 전주천동로 86-1 알지비게스트하우스,35.811075,127.151208,숙소
259,홍시게스트하우스,전라북도 전주시 완산구 은행로 51-11,35.815277,127.153067,숙소


In [247]:
res_ca.loc[0,"name"]

'양반가한정식'

In [250]:
li = ['어진박물관', '경기전', '전동성당','청연루','향교','오목대']

In [273]:
def dist_loc(i):
    pin_lat = hist_2.iloc[i,3]
    pin_long = hist_2.iloc[i,4]
    res = []
    for j in range(len(res_ca)):
        can_lat = res_ca.iloc[j,2]
        can_long = res_ca.iloc[j,3]
        dist_h = haversine((pin_lat,pin_long),(can_lat,can_long), unit='m')
        if dist_h < 150:
            dist_r = np.round(dist_h,2)
            res.append((res_ca.iloc[j,0],dist_r))        
    return res

candtoloc = [dist_loc(i) for i in range(len(hist_2))]

for i in range(len(candtoloc)):
    coun = len(candtoloc[i])
    print("{}:{}".format(hist_2.iloc[i,0],coun))
    
cound_li = [len(candtoloc[i]) for i in range(len(candtoloc))]
cound_li

3:38
4:19
5:17
6:23
7:37
8:50
11:24
12:16
13:3
14:2
15:3
16:16
17:31
18:36
19:52
20:33


[38, 19, 17, 23, 37, 50, 24, 16, 3, 2, 3, 16, 31, 36, 52, 33]